In [1]:
from tensorflow.keras.models import load_model

feature_extractor = load_model('extractor.h5')
print ("Loaded pre-trained convolution layers!")

Loaded pre-trained convolution layers!


In [2]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import optimizers
import numpy as np
from sklearn import preprocessing


(x_train, y_train), (x_test, y_test) = cifar10.load_data()
interested = [0, 1, 8, 9]

scrap = []
for idx, im in enumerate(x_train):
    if (y_train[idx][0] not in interested):
        scrap.append(idx)
        
x_train = np.delete(x_train, scrap, axis=0)
y_train = np.delete(y_train, scrap, axis=0)

enc = preprocessing.OneHotEncoder()
enc.fit(y_train)
y_train = enc.transform(y_train).toarray()

scrap = []
for idx, im in enumerate(x_test):
    if (y_test[idx][0] not in interested):
        scrap.append(idx)
x_test = np.delete(x_test, scrap, axis=0)
y_test = np.delete(y_test, scrap, axis=0)
y_test = enc.transform(y_test).toarray()

x_train = (x_train.astype('float32')) / 255.0
x_test = (x_test.astype('float32')) / 255.0

print (x_train.shape)
print (x_test.shape)

(20000, 32, 32, 3)
(4000, 32, 32, 3)


In [3]:
features = feature_extractor.predict(x_train)
features_test = feature_extractor.predict(x_test)

print (features.shape)
print (features_test.shape)

(20000, 4)
(4000, 4)


In [4]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import TensorBoard

class DeepAutoencoder:
    def __init__(self):
        self.encoder_model = None
        self.model = None
        return
    
    def build(self, input_dim, encoding_dim, opt):
        input_layer = Input(shape=(input_dim,))
        
        hidden_one = Dense(encoding_dim*4, activation='relu') (input_layer)
        hidden_two = Dense(encoding_dim*2, activation='relu') (hidden_one)
        encoder_output = Dense(encoding_dim, activation='relu') (hidden_two)
        self.encoder_model = Model(input_layer, encoder_output)
        
        hidden_three = Dense(encoding_dim*2, activation='relu') (encoder_output)
        hidden_four = Dense(encoding_dim*4, activation='relu') (hidden_three)
        decoder_output = Dense(input_dim, activation='sigmoid') (hidden_four)
        self.model = Model(input_layer, decoder_output)
        
        self.model.compile(optimizer=opt, loss='binary_crossentropy', )


        # This is the size of our encoded representations
#         encoding_dim = 32  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

#         # This is our input image
#         input_img = Input(shape=(784,))
#         # "encoded" is the encoded representation of the input
#         encoded = Dense(encoding_dim, activation='relu')(input_img)
#         # "decoded" is the lossy reconstruction of the input
#         decoded = Dense(784, activation='sigmoid')(encoded)

#         # This model maps an input to its reconstruction
#         self.model = Model(input_img, decoded)
#         # This model maps an input to its encoded representation
#         self.encoder_model = Model(input_img, encoded)
#         # This is our encoded (32-dimensional) input
#         encoded_input = Input(shape=(encoding_dim,))
#         # Retrieve the last layer of the autoencoder model
#         decoder_layer = self.model.layers[-1]
#         # Create the decoder model
#         decoder = Model(encoded_input, decoder_layer(encoded_input))
#         self.model.compile(optimizer='adam', loss='binary_crossentropy')
        return
    
    def load(self, model_file, encoder_model_file):
        self.encoder_model = load_model(encoder_model_file)
        self.model = load_model(model_file)
        return
    
    def train(self, train_input, train_output,
             val_input, val_output,
             epochs=50,
             batch_size=256,
             shuffle=True):
        tensorboard = TensorBoard(log_dir='./tf_logs_two', histogram_freq=0, write_graph=True, write_images=False)
        self.model.fit(train_input, train_output,
                      epochs=epochs, batch_size=batch_size,
                      shuffle=True,
                      validation_data=(val_input, val_output),
                      callbacks=[tensorboard]
                      )
        return
    
    def encoder_predict(self, test_input):
        return self.encoder_model.predict(test_input)
    
    def predict(self, test_input):
        return self.model.predict(test_input)
    
    def save(self, model_file, encoder_model_file):
        self.model.save(model_file)
        self.encoder_model.save(encoder_model_file)
        return

In [9]:
from tensorflow.keras import optimizers

pretrain_features_size = 4
target_dim_size = 32
autoencoder = DeepAutoencoder()
opt = optimizers.Adam(lr=0.0001)
autoencoder.build(pretrain_features_size, target_dim_size, opt)

In [10]:
autoencoder.train(features, features, features_test, features_test,
                 epochs=50,
                 batch_size=256,
                 shuffle=True)

Train on 20000 samples, validate on 4000 samples
Epoch 1/50
20000/20000 [==============================] - 2s 80us/sample - loss: 0.6687 - val_loss: 0.6358
Epoch 2/50
20000/20000 [==============================] - 0s 15us/sample - loss: 0.5694 - val_loss: 0.4939
Epoch 3/50
20000/20000 [==============================] - 0s 16us/sample - loss: 0.3983 - val_loss: 0.3099
Epoch 4/50
20000/20000 [==============================] - 0s 15us/sample - loss: 0.1902 - val_loss: 0.1330
Epoch 5/50
20000/20000 [==============================] - 0s 16us/sample - loss: 0.0600 - val_loss: 0.0715
Epoch 6/50
20000/20000 [==============================] - 0s 16us/sample - loss: 0.0230 - val_loss: 0.0592
Epoch 7/50
20000/20000 [==============================] - 0s 15us/sample - loss: 0.0130 - val_loss: 0.0572
Epoch 8/50
20000/20000 [==============================] - 0s 16us/sample - loss: 0.0094 - val_loss: 0.0575
Epoch 9/50
20000/20000 [==============================] - 0s 20us/sample - loss: 0.0078 - val_l

In [13]:
autoencoder.save('ae-complete.h5', 'ae-encoder.h5')
print ("Saved!")

Saved!
